In [ ]:
import os
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

cartoon = "DragonBall"
front_url = "https://fanfox.net/manga/dragon_ball/v39/c"
ch_start = 266
ch_end = 278
page_start = 1

print('start')

# 크롬 옵션 (헤드리스)
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")

time.sleep(3)

os.makedirs(cartoon, exist_ok=True)

for chapter in range(ch_start, ch_end + 1):
    chap_str = str(chapter).zfill(3)
    os.makedirs(f"{cartoon}/{chap_str}", exist_ok=True)

    stop_chapter = False

    for page in range(page_start, 200):

        retry_count = 0

        while retry_count < 5:

            base_url = front_url + chap_str + "/1.html#ipg" + str(page)
            url = base_url

            print(f"\n===== 접속: ipg : {url} (시도 {retry_count+1}/5)")

            try:
                driver = webdriver.Chrome(options=options)
                driver.get(url)

                sleep_time = random.uniform(3, 7)
                time.sleep(sleep_time)

                imgs = driver.find_elements(By.CLASS_NAME, "reader-main-img")

                print("찾은 이미지 개수:", len(imgs))

                # 챕터 종료 조건
                if (len(imgs) == 0) and (page > 5):
                    print("이미지 없음 → 챕터 종료")
                    driver.quit()
                    stop_chapter = True
                    break

                page_str = "img_" + str(page).zfill(3)

                gif_detected = False  # 🔥 추가

                for img in imgs:
                    img_url = img.get_attribute("src")

                    if img_url.startswith("//"):
                        img_url = "https:" + img_url

                    cookies = driver.get_cookies()
                    session = requests.Session()

                    for cookie in cookies:
                        session.cookies.set(cookie['name'], cookie['value'])

                    headers = {
                        "User-Agent": "Mozilla/5.0",
                        "Referer": url,
                        "Connection": "keep-alive",
                        "Accept": "image/webp,image/*,*/*;q=0.8"
                    }

                    img_data = session.get(img_url, headers=headers).content
                    filename = img_url.split("/")[-1].split("?")[0]
                    name, ext = os.path.splitext(filename)

                    # 🔥 gif 감지 시 페이지 재시도
                    if ext.lower() == '.gif':
                        print("gif 감지:", name)
                        retry_count += 1
                        gif_detected = True
                        driver.quit()

                        if retry_count >= 5:
                            print("gif 5회 발생 → 다음 페이지")
                        else:
                            print("페이지 재시도...")

                        time.sleep(3)
                        break  # for img 루프 탈출 → while 재시도

                    with open(f"{cartoon}/{chap_str}/{page_str}{ext}", "wb") as f:
                        f.write(img_data)

                    print("다운로드 완료:", filename)

                if gif_detected:
                    continue  # 🔥 while 재시도

                driver.quit()
                break  # 정상 완료 → 다음 페이지 이동

            except Exception as e:
                print("에러 발생:", e)
                retry_count += 1

                try:
                    driver.quit()
                except:
                    pass

                if retry_count >= 3:
                    print("3회 실패 → 다음 페이지")
                    break

                time.sleep(10)

        if stop_chapter:
            break

    print(f"챕터 {chap_str} 완료")
    time.sleep(5)

print("전체 완료")

start

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg1 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-000.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg2 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-001.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg3 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-002.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg4 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-003.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg5 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-004.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg6 (시도 1/5)
찾은 이미지 개수: 1
에러 발생: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c252/1.html#ipg6 (시도 2/5)
찾은 이미지 개수: 1
다운로드 완료: mdragon_ball_z_v022-0


===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg6 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-051.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg7 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-052.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg8 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-053.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg9 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-054.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg10 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-055.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg11 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-056.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c255/1.html#ipg12 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: cdragon_ball_z_v022-057.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c


===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg4 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-101.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg5 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-102.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg6 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-103.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg7 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-104.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg8 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-105.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg9 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-106.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c259/1.html#ipg10 (시도 1/5)
찾은 이미지 개수: 1
다운로드 완료: rdragon_ball_z_v022-107.jpg

===== 접속: ipg : https://fanfox.net/manga/dragon_ball/v38/c25